# Playground 6:  Segmentation workflows for "dots"-like structures

This notebook contains the workflows for Centrin-2, Desmoplakin, and PMP34, and serves as a starting point for developing a classic segmentation workflow for your data with dots-like shapes.

----------------------------------------

Cell Structure Observations:

* [Centrin-2](https://www.allencell.org/cell-observations/category/centrin)

* [Desmoplakin](https://www.allencell.org/cell-observations/category/desmoplakin)

* [PMP34](https://www.allencell.org/cell-observations/category/pmp34)

----------------------------------------

Key steps of the workflows:

* Min-max intensity normalization
* 2D Gaussian smoothing (slice-by-slice)
* 3D spot filter to detect dots
* Watershed for seperating falsely merged dots

In [ ]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects,  dilation, erosion, ball     # function for post-processing (size filter)
from skimage.segmentation import watershed

from aicssegmentation.core.utils import peak_local_max_wrapper
from skimage.measure import label
from scipy.ndimage import distance_transform_edt

## Loading the data

In [ ]:
FILE_NAME = '../../data/CETN2/original/3500001610_100X_20180112_1-Scene-10-P30-F05.czi'

reader = AICSImage(FILE_NAME) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

## Preview of the image

In [ ]:
N_CHANNELS = IMG.shape[2]
MID_SLICE = np.int(0.5*IMG.shape[3])

fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
if N_CHANNELS==1:
    ax.axis('off')
    ax.imshow(IMG[0,0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
else:
    for channel in range(N_CHANNELS):
        ax[channel].axis('off')
        ax[channel].imshow(IMG[0,0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

In [ ]:
#####################
structure_channel = 3
#####################

struct_img0 = IMG[0,0,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img0))

## Image segmentation

### Step 1: Pre-Processing

About selected algorithms and tuned parameters

* **Intensity normalization**: Parameter `intensity_scaling_param` has two options: two values, say `[A, B]`, or single value, say `[K]`. For the first case, `A` and `B` are non-negative values indicating that the full intensity range of the stack will first be cut-off into **[mean - A * std, mean + B * std]** and then rescaled to **[0, 1]**. The smaller the values of `A` and `B` are, the higher the contrast will be. For the second case, `K`>0 indicates min-max Normalization with an absolute intensity upper bound `K` (i.e., anything above `K` will be chopped off and reset as the minimum intensity of the stack) and `K`=0 means min-max Normalization without any intensity bound.

    * Parameter for Centrin-2:  `intensity_scaling_param = [8000]`
    * Parameter for Desmoplakin:  `intensity_scaling_param = [8000]`
    * Parameter for PMP34:  `intensity_scaling_param = [6000]`

These three all use Min-Max Normalization, just with slightly different absolute intensity upper bound. Using Min-Max normalization is meant to keep the full intensity profile near the brightest area, usually the center of the "balls" as in these structures. Such bright centers are important for the success of these workflows. Meanwhile, adding an absolute intensity upper bound is meant to be robust to imaging artifacts, like dead pixels. If there is one dead pixel in an image stack with extremely high intensity (way out of normal intensity range), the Min-Max normalization will squeeze the value of actual structures into a tiny range close to 0. So, we add an upper bound. For example, in Centrin-2, if a pixel has intensity value over 8000, this pixel will be treated as an outlier and reset as the minimum intensity of the stack. 

* **Smoothing** 

These three all use 2D gaussian smoothing with `gaussian_smoothing_sigma = 1`. Usually, 3D gaussian is used by default. But, here we apply 2D gaussian slice by slice, because when we do live imaging, these "ball"-shape structures could move visible amount (considering these "balls" themselves are tiny) during the time interval between two consecutive z slices. So, 3D guassian smoothing may further aggravate the subtle shift in consecutive z-slices. If your data do not have such problem, you can certainly try 3D gaussian by `image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)` with `gaussian_smoothing_sigma = 1`. To deal with very noisy data, you may consider increase `gaussian_smoothing_sigma` from 1 to a higher value, like 1.5 or 2.

In [ ]:
################################
intensity_scaling_param = [8000]
gaussian_smoothing_sigma = 1
################################

# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_slice_by_slice(struct_img, sigma=gaussian_smoothing_sigma)

In [ ]:
# quickly visualize the image after smoothing
view(single_fluorescent_view(structure_img_smooth))

### Step 2: Core Algorithm

#### step 2.1: Apply 3D Spot filter (S3)

* Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
    * `scale_x` is set based on the estimated radius of your target dots. For example, if visually the diameter of the dots is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have dots of very different sizes.  
    * `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf more dots and fatter segmentation, while larger `cutoff_x` could be less permisive and yield less dots and slimmer segmentation. 

* Parameter for Centrin-2:  `s3_param = [[1, 0.04]]`

* Parameter for Desmoplakin:  `s3_param = [[1, 0.012]]`

* Parameter for PMP34:  `s3_param = [[1,0.03]]`


In [ ]:
################################
## PARAMETERS for this step ##
s3_param = [[1, 0.04]]
################################

bw = dot_3d_wrapper(structure_img_smooth, s3_param)

In [ ]:
# view the segmentation result
viewer_bw = view(segmentation_quick_view(bw))
viewer_bw

##### After quickly visualizing the segmentation results, you can also visualize the segmentation and original image side by side
##### You may select an ROI to inspect the details

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_bw'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1:
view(seg_fluo_side_by_side(struct_img,bw,roi=['ROI',viewer_bw.roi_slice()]))

# Option 2: 
# view(seg_fluo_side_by_side(struct_img,bw,roi=['M',[570,370,730,440]]))

##### Is the segmentation satisfactory? Here are some possible criteria:

--------------------------
* Note: next step (2.2) can further cut falsely merged spots.
* Is there any spot should be detected but not? Try to reduce `cutoff_x`
* Is there any spot should not be detected but actually appear in the result? Try to increase `cutoff_x` or try a larger `scale_x`
* Is the segmented size of the spots fatter than it should be? Try to increase `cutoff_x` or try a smaller `scale_x`
* Is there any spot that should be solid but segmented as a ring? Try to increase `scale_x`
* Are you observing spots with very different sizes? Try multiple sets of `scale_x` and `cutoff_x` 
--------------------------

#### If the results are satisfactory, go to Step 2.2 directly; otherwise, try to tweak the parameters based on the above suggestions. 

Assumption: the segmentation result is saved in a variable named `bw`.

#### Step 2.2: Watershed for cutting falsely merged spots

This step may take 2~3 minutes or longer, depending on the number of dots in your images. 

In general, no parameter tuning is needed in this step. The only parameter is `minArea=4`, which is the parameter for size thresholding in post-processing. In practice, when we know an object whose size is smaller than `minArea` will be removed eventually, we can just remove them before doing watershed to cut falsely merged cells. This is only meant to avoid unnecessary computation. 

In [ ]:
# watershed
minArea = 4
Mask = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False) 
Seed = dilation(peak_local_max_wrapper(struct_img,label(Mask)), selem=ball(1))
Watershed_Map = -1*distance_transform_edt(bw)
seg = watershed(Watershed_Map, label(Seed), mask=Mask, watershed_line=True)

#### Step 3: Post-Processing 

In [ ]:
################################
## PARAMETERS for this step ##
minArea = 4 
################################

seg = remove_small_objects(seg>0, min_size=minArea, connectivity=1, in_place=False)

## Result inspection

In [ ]:
viewer_final = view(segmentation_quick_view(seg))
viewer_final

### You can also focus your inspection on a small ROI

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_final'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1: 
view(seg_fluo_side_by_side(struct_img, seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2: 
#view(seg_fluo_side_by_side(struct_img, seg, roi=['M',[267,474, 468, 605]]))

### You can also inspect the same ROI on the effect of final cutting step (the same ROI as above)

In [ ]:
# Option 1 for ROI selection:
view(seg_fluo_side_by_side(bw, seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2 for ROI selection:
#view(seg_fluo_side_by_side(bw, seg, roi=['M',[570,370,730,440]]))
# left is the final version
# right is the version before cutting

### You may also physically save the segmentation results into a .tiff image

In [ ]:
seg = seg >0
out=seg.astype(np.uint8)
out[out>0]=255
writer = OmeTiffWriter('../../data/CETN2/result/3500001610_100X_20180112_1-Scene-10-P30-F05_test_segmentation.tiff')
writer.save(out)